In [22]:
#!pip install torchio
#!pip install nilearn

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
# import torchio
import os
import nilearn
from sklearn.model_selection import train_test_split
from skimage.transform import resize
import torch.nn as nn


%matplotlib inline

In [23]:
#getting labels

targets = pd.read_csv('D:/Target_and_data/Target.csv')
y = pd.get_dummies(targets, columns=['Gender'])['Gender_M'].values

In [24]:
X=np.load('D:/Target_and_data/X.npy')
X.shape

(517, 58, 70, 58)

In [27]:
X = X[:,None,:,:,:]

In [28]:
X.shape

(517, 1, 58, 70, 58)

In [29]:
class MriData(torch.utils.data.Dataset):
    def __init__(self, X, y):
        super(MriData, self).__init__()
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #may be that's bad idea to split using train_test_split. Instead of it - use indexes of data
#del X, y #deleting for freeing space on disc

In [31]:
train_dataset = MriData(X_train, y_train)
test_dataset = MriData(X_test, y_test)
#del X_train, X_test, y_train, y_test. #deleting for freeing space on disc

In [32]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=28, shuffle=True) 
val_loader = torch.utils.data.DataLoader(test_dataset, batch_size=28, shuffle=False)

In [33]:
class MyFirstAE(nn.Module):
    def __init__(self, net):
        super(MyFirstAE, self).__init__()
        self.net = net
    
    def forward(self, x):
        """
        Take a mini-batch as an input, encode it to the latent space and decode back to the original space
        x_out = decoder(encoder(x))
        :param x: torch.tensor, (MB, x_dim)
        :return: torch.tensor, (MB, x_dim)
        """
        return self.net(x)

In [34]:
def image_size_conv(im_size,conv_kernel, conv_stride, padding):
    return (im_size-conv_kernel+2*padding)/conv_stride +1
def image_size_maxpool(im_size, pool_kernel,pool_stride):
    return (im_size-pool_kernel)/pool_stride +1

In [35]:
im_size = np.array([SCALE[0],SCALE[1],SCALE[2]])

conv_kernel_1 = 3
pool_kernel_1 = 2
channels_1 = 5
conv_stride_1 = 1
pool_stride_1 = 2
padding_1=0
im_size = image_size_conv(im_size,conv_kernel_1, conv_stride_1, padding_1)
im_size = image_size_maxpool(im_size, pool_kernel_1,pool_stride_1)

conv_kernel_2 = 3
pool_kernel_2 = 2
channels_2 = 8
conv_stride_2 = 1
pool_stride_2 = 2
padding_2 = 0
im_size = image_size_conv(im_size,conv_kernel_2, conv_stride_2, padding_2)
im_size = image_size_maxpool(im_size, pool_kernel_2,pool_stride_2)

conv_kernel_3 = 3
pool_kernel_3 = 2
channels_3 = 16
conv_stride_3 = 1
pool_stride_3 = 2
padding_3 = 0
im_size = image_size_conv(im_size,conv_kernel_3, conv_stride_3, padding_3)
im_size = image_size_maxpool(im_size, pool_kernel_3,pool_stride_3)
im_size = im_size.astype('int')

model =  nn.Sequential(
    nn.Conv3d(in_channels=1,out_channels=channels_1,kernel_size=conv_kernel_1),
    nn.BatchNorm3d(channels_1),
    nn.ReLU(),
    nn.MaxPool3d(kernel_size=pool_kernel_1),
    
    nn.Conv3d(in_channels=channels_1,out_channels=channels_2,kernel_size=conv_kernel_2),
    nn.BatchNorm3d(channels_2),
    nn.ReLU(),
    nn.MaxPool3d(kernel_size=pool_kernel_2),
    
    
    nn.Conv3d(in_channels=channels_2,out_channels=channels_3,kernel_size=conv_kernel_3),
    nn.BatchNorm3d(channels_3),
    nn.ReLU(),
    nn.MaxPool3d(kernel_size=pool_kernel_3),
    
    nn.Flatten(),
    nn.Linear(np.prod(im_size)*channels_3,500),
    torch.nn.ReLU(),
    nn.Linear(500,100),
    torch.nn.ReLU(),
    nn.Linear(100,2),
    )

In [36]:
def train(epochs,device, net, criterion, optimizer, train_loader, val_loader,scheduler=None, verbose=True, save_dir=None):
    net.to(device)
    for epoch in range(1, epochs+1):
        net.train()
        for X, y in train_loader:

            X = X.to(device)
            y = y.to(device)
#             print(y.shape)
            out = net(X)
#             print(out.shape)
            loss = criterion(out,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # >>> your solution here <<<
        

        net.eval()
        for X, y in val_loader:
            # Compute the validation loss
            X = X.to(device)
            y = y.to(device)
            out = net(X)
            val_loss = criterion(out,y)
        if scheduler is not None:
            scheduler.step()
        freq = max(epochs//20,1)
        if verbose and epoch%freq==0:
            print('Epoch {}/{} || Loss:  Train {:.4f} | Validation {:.4f}'.format(epoch, epochs, loss.item(), val_loss.item()))

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = MyFirstAE(model) 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=(15,25 ), gamma=.1)


In [38]:
epochs = 50
train(epochs,device, net, criterion, optimizer, train_loader, val_loader)

Epoch 2/50 || Loss:  Train 0.3448 | Validation 0.3694
Epoch 4/50 || Loss:  Train 0.5481 | Validation 0.1538
Epoch 6/50 || Loss:  Train 0.3455 | Validation 0.0954
Epoch 8/50 || Loss:  Train 0.0972 | Validation 0.0684
Epoch 10/50 || Loss:  Train 0.2895 | Validation 0.0713
Epoch 12/50 || Loss:  Train 0.3372 | Validation 0.0466
Epoch 14/50 || Loss:  Train 0.0718 | Validation 0.0348
Epoch 16/50 || Loss:  Train 0.0256 | Validation 0.0645
Epoch 18/50 || Loss:  Train 0.0077 | Validation 0.1506
Epoch 20/50 || Loss:  Train 0.0052 | Validation 0.0571
Epoch 22/50 || Loss:  Train 0.0020 | Validation 0.0424
Epoch 24/50 || Loss:  Train 0.0018 | Validation 0.0882
Epoch 26/50 || Loss:  Train 0.0003 | Validation 0.0396
Epoch 28/50 || Loss:  Train 0.0004 | Validation 0.0398
Epoch 30/50 || Loss:  Train 0.0007 | Validation 0.0484
Epoch 32/50 || Loss:  Train 0.0003 | Validation 0.0219
Epoch 34/50 || Loss:  Train 0.0001 | Validation 0.0301
Epoch 36/50 || Loss:  Train 0.0000 | Validation 0.0161
Epoch 38/50 ||